# IMDB Movies Dataset
In this exercise, we will explore data from IMDB (https://www.imdb.com/; https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset#movie_metadata.csv), a movies website where people can rank movies, read and write reviews, etc.

Below is some basic code for reading and visualizing the data. 

Data description:

- color: is the movie in color or black and white
- director_name: name of the director of the movie
- duration: the duration of the movie in minutes
- director_facebook_likes: how many likes does the movie director have on facebook
- gross: the gross income of the movie
- genres: the genre of the movie (e.g., action)
- movie_title: the name of the movie
- num_voted_users: number of IMDB users who rated the movie
- cast_total_facebook_likes: how many likes do the cast (actors of the movie) have on facebook in total
- budget: the budget of the movie
- title_year: the year in which the movie came out
- imdb_score: the average rating of the movie on imdb (scale of 1-10)
- movie_facebook_likes: how many likes does the movie have on facebook

In [ ]:
import pandas as pd  # pandas is a package we will use to work with data tables
import matplotlib.pyplot as plt  # matplotlib is a package for plotting data
import seaborn as sns  # seaborn is also a package for plotting data, built on top of matplotlib
import numpy as np  # numpy is a package for working with numerical data

sns.set_context("talk") # seaborn theme that makes plots clearer when presenting
sns.set_palette("viridis") # change color pallette to the recommended Viridis

movies_df = pd.read_csv('movie_metadata.csv')  # reads dataset to a dataframe 
display(movies_df.head())  # prints first 5 rows of dataframe
movies_df.info()

How many observations do we have? 
Do we have missing values? In which variables?

In [ ]:
# let's describe the data (separately for numeric and categorical variables)
display(movies_df.describe()) # describes only numeric variables
movies_df.describe(include=[object]) # descirbe only cateogrical variables (actually "object" type variables)

What is the average number of IMDB votes for a movie in this dataset? 
What is the median number?

How many of Steven Spielberg's movies are in this dataset?

## Explore distributions of variables

### Numeric variables

Run the code below to see the distribution of the budget of movies. 

Let's play with the parameter "bins", which controls the number of bins in the histogram. Can you make different observations as you change it?

**Challenge for home:** Try to look at the distributions of other (numeric) variables that you are curious about. E.g., what does the distribution of movie ratings looks like? of movie durations?

In [ ]:
ax=sns.displot(data=movies_df, x="budget", bins=20)

### categorical variables

Run the code below to see the distribution of movie genres.

Would it be wise to plot distributions of other categorical variables in this dataset? Hint: look at the summary statistics table above.

**Challenge for home:** Try to flip the chart 90 degress counter-clockwise, and make sure the labeles are readable

In [ ]:
sns.catplot(data=movies_df, y='genres', kind='count')

Clearly, there are very few movies in several generes. Let's change their category to "other"

In [ ]:
# make a list of categories we want to change
genres_to_remove = ['Sci-Fi','Romance','Family','Western','Musical','Thriller']
# change the value in 'genres' of observations whose genre is in the list to 'other'
movies_df.loc[movies_df['genres'].isin(genres_to_remove),'genres'] = 'Other'
# replot the cell above
sns.catplot(data=movies_df, y='genres', kind='count')

## Explore relationship between variables

Run the code below to see a scatter plot showing the relationship between the movie's budget and its income. Does there seem to be a clear pattern? Is there anything weird in the data?

**Challenge for home:** Modify the code to explore other potential relationships between variables:

- Do you think there is a strong correlation between the budget of a movie and its rating?
- Do you think there is a strong correlation between the gross income of a movie and its rating?
- Do you think there is a strong correlation between the facebook likes of a movie and its rating?
- Do you think there is a strong correlation between the facebook likes of a movie and the facebook likes of its director?


In [ ]:
sns.relplot(data=movies_df, x='budget', y='gross', kind='scatter')

In [ ]:
# checking the correlation 
movies_df['budget'].corr(movies_df['gross'])

### Outliers

The budget of some movies in the data seems too high. Let's print out the names of these movies

In [ ]:
outliers = movies_df.loc[movies_df['budget'] > 3e+08]
outliers

Let's remove these from the data

In [ ]:
movies_df = movies_df.loc[movies_df['budget'] <= 3e+08]

These outliers indicate that the data includes errors. <br>
Let's try to find another variable with suspicious values and check if they make sense. For example, let's check the distribution of cast_total_facebook_likes

In [ ]:
# the rug argument makes sure we can see outliers plotted.
sns.displot(data=movies_df, x='cast_total_facebook_likes',kind='kde', rug=True) 

In [ ]:
outliers = movies_df.loc[movies_df['cast_total_facebook_likes'] >=200000]
outliers

These also seem unreasonable, but not as unreasonable as the *budget* outliers above. Normally, we'd investigate more, but for now let's remove them.

**Challenge for home:** Check for other possible outliers, and remove them if necessary.

In [ ]:
movies_df = movies_df.loc[movies_df['cast_total_facebook_likes'] <= 200000]

## Make comparisons between movie genres

Run the code below to see the a bar chart showing the average income for movies of different genres. What do you observe? What does this mean for movie producers? Does this mean that if they want to make more money, they should focus on specific genres?

**Challenge for home:** Modify the code to explore other comparisons between genres, e.g.:

- Movies from which genres cost more to produce?
- Movies from which genres have get higher ratings?



In [ ]:
p = sns.catplot(data=movies_df, x='genres', y='gross', kind='bar',height=7, aspect=2) # aspect controls the ratio of width to height of the plot
p.set_xticklabels(size = 14) # make font size of tick lables on x axis smaller so they dont go over one another

## Make comparisons between movie genres - part 2

Run the code below to see the a boxplot showing the average ratings for movies of different genres. What can you infer that you could not see in the bar chart above?

Modify the code to look at boxplots for other comparisons (as above).


In [ ]:
p = sns.catplot(data=movies_df, x='genres', y='imdb_score', kind='box', showmeans=True, height=7, aspect=2) # note the showmeans argument
p.set_xticklabels(size = 14)

## Trends over time

We want to see how the average revenue of a movie changes over time. First, we need to find the average revenue of a movie for each year in the data. <br>
Yet, there are years with very few movies, and the average will not be informaive. So we will also remove years that have fewer than 50 movies.

Run the code below to create a new dataframe with the average revenue per year

In [ ]:
grpby_year = movies_df.groupby(['title_year']) # first, we create a groupby object by year
agg_types = {
    'gross': ['mean','count']
}

df_year_gross = grpby_year.agg(agg_types)

df_year_gross # checking to see we have what we want...


In [ ]:
# the groupby object automatically uses year for indexing, so we will not be able to use it as a variable
# reset_index allows us to use year again as a variable
df_year_gross.reset_index(inplace=True) # inplace asks to do the resetting on the calling df
df_year_gross

# Alternatively, we could have set as_index=False when we created the groupby object

In [ ]:
# let's rename the columns to make things easier
display(df_year_gross.columns.values)
df_year_gross.columns = ['title_year', 'mean_gross', 'count_gross']
df_year_gross.columns.values

# alternatively, we could have used "named aggregations" when creating the new dataframe

In [ ]:
# now we can remove years with fewer than 50 movies
df_year_gross = df_year_gross.loc[df_year_gross['count_gross'] > 50]
df_year_gross

Run the code below to plot a line chart of the trend in average revenue for the years we have left in the data

Then, try to change the code above and below to llok at other interesting variables over time

In [ ]:
sns.relplot(data=df_year_gross, x='title_year', y='mean_gross', kind='line')

## Feature engineering

Creating new features very much depends on the questions we want to ask regarding the movies in this dataset. Let's say we're investigating the likes on Facebook for the movie, its cast, and its director. Here are a few examples of features we can engineer.

- If we want to say something regarding the total Facebook exposure of the movie, we can sum the three Facebook-like variables
- If we want to ask whether a movie is more or less popular than its director, we can compute the ratio of the movie Facebook likes to the director's Facebook likes. What is another feature you'd design for such question?
- If we believe the number of likes is strongly related to the popularity of Facebook in the year in which the movie came out, we can design an indicator variable for years>2010 (when Facebook had over 500,000,000 users). What would be another feature you may use here?
- It is possible the movie or director has no fan page on Facebook, so the number of likes would be 0. Perhaps we need an indicator for 0 likes for each such variable.

Try to create these, and other possibly interesting, features.